# Scrape Race Results

In [ ]:
from bs4 import BeautifulSoup
import requests, re
import pandas as pd

In [ ]:
BASEURL = 'http://raceresults.co.za/'
URL = BASEURL + 'output_full_results.php'

## Retrieve Race Page

There are also `sex` and `C1` components in the full POST payload but they are not necessary!

In [ ]:
payload = {
    'events': 'event_name=Spirit of Flight (AGN)&distance=10km&date=2017-09-02'
}
headers = {
    'User-Agent': 'Lynx/2.8.6rel.5 libwww-FM/2.14'
}

In [ ]:
with requests.post(URL, headers=headers, data=payload) as r:
    soup = BeautifulSoup(r.text, 'lxml')

In [ ]:
soup

## Race Metadata

Check that we've retrieved a results page.

In [ ]:
soup.title

Retrieve race name and number of finishers.

In [ ]:
[tag.text for tag in soup.select('body p.headings2')]

## Process Table

In [ ]:
table = soup.find("table", id = 'results_full')

In [ ]:
table.find_all("tr")

This is an example of page with poor structure! The rest of the results table is not actually divided into explicity rows. Take a look at the HTML: only the first row in the table has `<tr>` and `</tr>` tags.

We can still handle this though!

In [ ]:
table.find_all('td')

### Column Headings

In [ ]:
table_names = [re.sub(" ", "_", td.text.lower()) for td in table.find('tr').find_all('td')]

### Table Contents

Select all of the cells except the first row.

In [ ]:
table_data = [td.text for td in table.find_all('td')[7:]]
len(table_data)

In [ ]:
table_data

Now we need to wrap that into nested lists.

In [ ]:
table_data = [table_data[i:i+7] for i in range(0, len(table_data), 7)]
table_data

And convert that into a data frame.

In [ ]:
df = pd.DataFrame(table_data, columns=table_names)

In [ ]:
df

## Link to Next Page

Find the **Next** link at the bottom of the page.

In [ ]:
next = soup.find('a', text='Next')

In [ ]:
next['href']

We could iterate through the pages using something like this:

In [ ]:
while True:
    #
    # You'd need to replace this with the code to scrape each page and accumulate the data.
    #
    print('Processing next page...')
    #
    next = soup.find('a', text='Next')
    #
    if next is None:
        break
    else:
        next_url = BASEURL+next['href']
        soup = BeautifulSoup(requests.get(next_url, headers=headers).text, 'lxml')